# Battle of the Neighborhood

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import folium
from pandas.io.json import json_normalize
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# Week -3
## Week -3: question 1

### Download the data from wikipedia

Online link for Toronto https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [5]:
#Collecting the data from Wikipedia
wiki = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header = 0)

#Adjusting the table
Toronto_raw = wiki[0]

print(Toronto_raw.shape)
Toronto_raw.head()

(180, 3)


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [6]:
#Renanaming the columns
Toronto_raw.rename(columns = {'Postal Code':'PostalCode', "Neighbourhood": "Neighborhood"}, inplace = True)
Toronto_raw.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
#Dropping the 'Not assigned'
Toronto_raw.drop(Toronto_raw[Toronto_raw.Borough == 'Not assigned'].index, inplace=True)
Toronto_raw.reset_index(inplace=True, drop=True)
Toronto_raw.head(11)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [18]:
#Combinig Neighborhood with same postal code
Toronto_raw = Toronto_raw.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
print('Shape of the Data Frame:', Toronto_raw.shape)
Toronto_raw.head()

Shape of the Data Frame: (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Week - 3: question 2

In [23]:
#readin the csv data
LatLong = pd.read_csv('http://cocl.us/Geospatial_data')
#Renaming the columns to match the previous table
LatLong.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
print('Shape of the Data Frame:', LatLong.shape)
LatLong.head()

Shape of the Data Frame: (103, 3)


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
#Mearging the two table
Toronto_merge = pd.merge(Toronto_raw, LatLong, on=['PostalCode'])
print('Shape of the Data Frame:', Toronto_merge.shape)
Toronto_merge.head()

Shape of the Data Frame: (103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
